In [1]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer

/home/dhlee347/anaconda3/envs/mkt/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model_name = "/w/exp/mkt/model_8.8B_BI_MT_02-4th/checkpoint-20"

config = AutoConfig.from_pretrained(model_name)
config.gradient_checkpointing = True
config.use_cache = False

tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(model_name, config=config).to("cuda")
model.resize_token_embeddings(len(tokenizer))

Embedding(102400, 4096)

In [14]:
prompt= """다음의 상품을 홍보하는 창의적인 마케팅 문구를 작성해 보세요.
[회사] 동아오츠카
[카테고리] 식품, 음료, 주류
[분류]
[상품명] 오로나민C
[마케팅 문구] """

In [15]:
from random import random

def get_label(gen_ids):
    generated = tokenizer.decode(gen_ids)
    start_index = generated.find("[마케팅 문구]") + 9
    label = generated[start_index:].replace("[EOS]", "").strip()
    return label

input_ids = tokenizer.encode(prompt, return_tensors='pt').to("cuda")

for i in range(10):
    temp = random()*2.0
    gen_ids = model.generate(input_ids,
                             max_length=256,
                             repetition_penalty=2.0,
                             do_sample=True,
                             temperature=temp,
                             pad_token_id=tokenizer.pad_token_id,
                             eos_token_id=tokenizer.eos_token_id,
                             bos_token_id=tokenizer.bos_token_id,
                             use_cache=True)
    print(f"{'-'*70}Temperature={temp:.2}{'-'*30}")
    print(get_label(gen_ids[0]))

----------------------------------------------------------------------Temperature=1.8------------------------------
오늘 나는 에너지 뿜뿜하는데 나보다도 땀많은 박보결 체력방전
----------------------------------------------------------------------Temperature=0.45------------------------------
쨍하고 해뜰날은 아직 멀었나요? 피곤한 당신에게 오로나민C가 필요합니다
----------------------------------------------------------------------Temperature=1.2------------------------------
비타민으로 채우기 힘겨웠던 무더위 속 나의 활력 비타민
----------------------------------------------------------------------Temperature=2.0------------------------------
비타민 C 10 배로 활력 있게 살아나는 너도나도 C 타임!
"이런 내 사랑 이런 남자 또 없습니다"" 비타민처럼 내 활력은 널 향해있어 C타임대."
----------------------------------------------------------------------Temperature=1.7------------------------------
어떤 맛인들 어때. 하나 되는 세상으로
----------------------------------------------------------------------Temperature=0.28------------------------------
비타민C가 피로회복에 좋다는 건 누구나 아는 사실
---------------------------------------------------------------